# Demo prototypes of IN-CORE services
This notebook shows how to use prototypes of IN-CORE services by using pyincore library. This demo shows how you can use IN-CORE services with other python libraries

In [ ]:
from pyincore import FragilityService, DataService, HazardService, ComputeDamage
from pyincore import PlotUtil
from pyincore import InventoryDataset
import pprint
pp = pprint.PrettyPrinter(indent=4)

## Using Data Service (http://localhost:8080/data/api/datasets)
* __get_datasets(...)__: retriving metadata with a query (by data type or/and by title)
* __get_dataset_metadata(...)__: retrieving metadata of dataset
* __get_dataset__: retrieving files of dataset

### Find dataset that contains "hospital" in the title

In [ ]:
DataService.get_datasets('http://localhost:8080',title="hospital")

### Download the files of Hospital dataset

In [ ]:
localfile = DataService.get_dataset('http://localhost:8080', '59f3617de1b2b828641a012a')
buildings = InventoryDataset(localfile)

### IN-CORE web service can work with other popular python library (e.g. GeoPandas) 
The following shows plotting the dataset from IN-CORE web service by using GeoPandas

In [ ]:
import geopandas as gpd
gdf = gpd.GeoDataFrame.from_features(buildings.inventory_set)
#hospital= gpd.read_file('data/Hospitals1228337472406', layer='all_bldgs_ver5_WGS1984')

In [ ]:
%matplotlib inline
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 8, 6
gdf.plot()

In [ ]:
## show Top 5 rows of attribute table of the Hospital dataset
gdf.head()

### Showing hospital dataset (IN-CORE service) on Open Street Map by using GeoPandas and Folium

In [ ]:
import folium

m = folium.Map(location=[-89.88505, 35.220586])
folium.GeoJson(gdf.to_json(), name='hospital').add_to(m)
ext = gdf.total_bounds
m.fit_bounds([[ext[1], ext[0]], [ext[3], ext[2]]])
m

## Using Glossary Service (https://resilience-glossary.ncsa.illinois.edu)
* __get_term(...)__: retriving a term from the glossary service

In [ ]:
from pyincore import GlossaryService
term = GlossaryService.get_term("https://resilience-glossary.ncsa.illinois.edu", "Q446")
term.description

# Building Damage Estimation by using Data Service, Fragility Service, and Hazard Service 
The following examples show how building damage can be computed by using Data Service, Fragility Service and Hazard Service. 

### Step 1. Retrive a building from Hospital dataset

In [ ]:
building = next(iter(buildings.inventory_set))

### Step 2. Retrive a Fragility Curve Set
* The Fragility Service finds a fragility curve set with the structure attribute of the building

In [ ]:
fragility_set = FragilityService.map_fragility('http://localhost:8080', building, "Non-Retrofit Fragility ID Code")

## print the retrived fragility curve set
fragility_set

In [ ]:
## Plot the fragility curve set
## plot the retrived f
import matplotlib.pyplot as plt

for curve in fragility_set['fragilityCurves']:
    x,y = PlotUtil.get_x_y(curve['curveType'],curve['median'],curve['beta'])
    plt.plot(x,y, label=curve['description'])
plt.xlabel(fragility_set['demandType']+" ("+fragility_set['demandUnits']+")")
plt.legend()
plt.show()

### Step 3: Getting Hazard Value at the location of the building
* Scenario earthquake has been created at the service with the following parameters:
    * AtkinsonBoore 1995
    * Epicenter (lat/lon): 35.927, -89.919
    * Magnitude: 7.9
    * Depth: 10.0
* Plot the scenario earthquake with sampling data from Hazard Service

In [ ]:
import numpy as np
from scipy.interpolate import griddata
x, y, z = HazardService.get_hazard_value_set("http://localhost:8080", "59f8e10004c01e79c4a67a88", "PGA", "g", [[-90.3099,34.9942],[-89.6231,35.4129]], 0.01696)
grid_x, grid_y = np.mgrid[min(x):max(x):100j, min(y):max(y):100j]
grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')
contour_filled=plt.contourf(grid_x, grid_y, grid_z, 20, cmap="gist_rainbow_r")
plt.colorbar(contour_filled)
plt.show()

* __Getting a hazard value at the location of the building by using Hazard Service__

In [ ]:
latlon = building['geometry']['coordinates']
haz_val = HazardService.get_hazard_value("http://localhost:8080", "59f8e10004c01e79c4a67a88", "PGA", "g", latlon[1], latlon[0])
haz_val

### Step 4. Compute limit states with the fragility curve sets and the hazard value

In [ ]:
damage = ComputeDamage.calculate_damage_json2(fragility_set, haz_val)
damage

## This demonstrates that IN-CORE web services can be used with popular python libraries (Statisitcs, Data analysis etc) 

In [ ]:
import numpy as np
from scipy import ndimage
def heatmap(d, bins=(100,100), smoothing=1.3, cmap='jet'):
    def getx(pt):
        return pt.coords[0][0]

    def gety(pt):
        return pt.coords[0][1]

    x = list(d.geometry.apply(getx))
    y = list(d.geometry.apply(gety))
    heatmap, xedges, yedges = np.histogram2d(y, x, bins=bins)
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]

    logheatmap = np.log(heatmap)
    logheatmap[np.isneginf(logheatmap)] = 0
    logheatmap = ndimage.filters.gaussian_filter(logheatmap, smoothing, mode='nearest')
    
    plt.imshow(logheatmap, cmap=cmap, extent=extent)
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
localfile_res3 = DataService.get_dataset('http://localhost:8080', '59f36185e1b2b828641a0189')
res3 = InventoryDataset(localfile_res3)

In [ ]:
res3_gdf = gpd.GeoDataFrame.from_features(res3.inventory_set)
heatmap(res3_gdf, bins=100, smoothing=1.5)
